In [1]:
import pandas as pd
import numpy as np
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

In [2]:
# Load the CSV files into DataFrames
df_intake = pd.read_csv("Nutrient_Intake_Recommendations_Formalized.csv")
df_recipe_cost = pd.read_csv("Recipe_Cost_Formalized.csv")
df_recipe_nutrition = pd.read_csv("Recipe_Nutritional_Value_Formalized.csv")

Prediction for one meal

In [3]:
# Extract required data
time_available = 60*1.3  # Example time available (in minutes)
# nutrient_requirements = {
#     'Calcium': 1000,
#     'Carbohydrate': 351,
#     'Cholesterol': 0,
#     'Energy': 3119
# }

# Define the nutrient requirements dictionary from the df_intake DataFrame
nutrient_requirements = dict(zip(df_intake['Nutrient'], df_intake['Quantity']))

# Define decision variables
recipes_to_produce = LpVariable.dicts("Recipe", df_recipe_cost['Name'], lowBound=0, cat='Integer')

# Define the optimization problem
prob = LpProblem("Recipe_Production", LpMaximize)

# Objective function: maximize food preference produced
prob += lpSum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Preference'].values[0] * recipes_to_produce[recipe] for recipe in df_recipe_cost['Name'])

# Constraint 1: time cost
prob += lpSum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * recipes_to_produce[recipe] for recipe in df_recipe_cost['Name']) <= time_available

# Constraint 2: nutritional requirements
for nutrient in nutrient_requirements:
    prob += lpSum(df_recipe_nutrition.loc[df_recipe_nutrition['Name'] == recipe, nutrient].values[0] * recipes_to_produce[recipe] for recipe in df_recipe_nutrition['Name']) >= nutrient_requirements[nutrient]/3

# Solve the optimization problem
prob.solve()

# Print the results
print("Status:", prob.status)
print("Objective:", round(prob.objective.value(), 2))

# for recipe in df_recipe_cost['Name']:
#     quantity = recipes_to_produce[recipe].value()
#     if quantity > 0:
#         print(f"Quantity of {recipe}: {round(quantity, 2)}")

# Initialize an empty list to store the quantities
alpha = []

# Iterate through all recipes
for recipe in df_recipe_cost['Name']:
    # Retrieve the quantity from the optimization results
    quantity = recipes_to_produce[recipe].value()
    # Append the quantity to the alpha list
    alpha.append(round(quantity, 2))

# Print the whole array of quantities
#print("Quantity array (alpha):", alpha)

output = dict(zip(list(df_recipe_cost['Name']), alpha))

# Filter out dishes with a number to make of 0.0
output = {dish: quantity for dish, quantity in output.items() if quantity > 0}
# Calculate time spent for each meal
time_spent = sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * alpha[i] for i, recipe in enumerate(df_recipe_cost['Name']))
# Print out time spent for each meal
print("Time spent:", round(time_spent, 2), "minutes")


output

Status: 1
Objective: 9.0
Time spent: 75.0 minutes


{'Broccolini Quinoa Pilaf': 1.0,
 'Vanilla Cream Cakes, Easy and Fluffy Holiday Cakes': 1.0}

Prediction for three meals. Namely breakfast, lunch and dinner

In [4]:
# Define the time available for each meal
time_breakfast = 60*0.5  # Example time available for breakfast (in minutes)
time_lunch = 60*1  # Example time available for lunch (in minutes)
time_dinner = 60*2  # Example time available for dinner (in minutes)

# Define the nutrient requirements dictionary from the df_intake DataFrame
nutrient_requirements = dict(zip(df_intake['Nutrient'], df_intake['Quantity']))

# Define decision variables for each meal
recipes_to_produce_breakfast = LpVariable.dicts("Recipe_Breakfast",df_recipe_cost['Name'], lowBound=0, cat='Integer')
recipes_to_produce_lunch = LpVariable.dicts("Recipe_Lunch", df_recipe_cost['Name'], lowBound=0, cat='Integer')
recipes_to_produce_dinner = LpVariable.dicts("Recipe_Dinner", df_recipe_cost['Name'], lowBound=0, cat='Integer')

# Define the optimization problem
prob = LpProblem("Recipe_Production_for_day", LpMaximize)

# Objective function: maximize food preference produced
prob += lpSum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Preference'].values[0] * 
              (recipes_to_produce_breakfast[recipe] + recipes_to_produce_lunch[recipe] + recipes_to_produce_dinner[recipe])
              for recipe in df_recipe_cost['Name'])

# Constraint 1: time cost for each meal
prob += lpSum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * recipes_to_produce_breakfast[recipe] for recipe in df_recipe_cost['Name']) <= time_breakfast
prob += lpSum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * recipes_to_produce_lunch[recipe] for recipe in df_recipe_cost['Name']) <= time_lunch
prob += lpSum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * recipes_to_produce_dinner[recipe] for recipe in df_recipe_cost['Name']) <= time_dinner

# Constraint 2: nutritional requirements: we want that combination all three meals together satisfy the nutritional requirements
for nutrient in nutrient_requirements:
     prob += lpSum(df_recipe_nutrition.loc[df_recipe_nutrition['Name'] == recipe, nutrient].values[0] *
                   (recipes_to_produce_breakfast[recipe] + recipes_to_produce_lunch[recipe] + recipes_to_produce_dinner[recipe])
                  for recipe in df_recipe_nutrition['Name']) >= nutrient_requirements[nutrient]

# Solve the optimization problem
prob.solve()

# Print the results
print("Status:", prob.status)
print("Objective:", round(prob.objective.value(), 2))

# Initialize an empty list to store the quantities
alpha_b = []
alpha_l = []
alpha_d = []


for recipe in df_recipe_cost['Name']:
    # Retrieve the quantity from the optimization results
    quantity_b = recipes_to_produce_breakfast[recipe].value()
    quantity_l = recipes_to_produce_lunch[recipe].value()
    quantity_d = recipes_to_produce_dinner[recipe].value()
    
    # Append the quantity to the alpha list
    alpha_b.append(round(quantity_b, 2))
    alpha_l.append(round(quantity_l, 2))
    alpha_d.append(round(quantity_d, 2))

output_b = dict(zip(list(df_recipe_cost['Name']), alpha_b))
output_l = dict(zip(list(df_recipe_cost['Name']), alpha_l))
output_d = dict(zip(list(df_recipe_cost['Name']), alpha_d))

# Filter out dishes with a number to make of 0.0
output_b= {dish: quantity for dish, quantity in output_b.items() if quantity > 0}
output_l = {dish: quantity for dish, quantity in output_l.items() if quantity > 0}
output_d = {dish: quantity for dish, quantity in output_d.items() if quantity > 0}

# Calculate time spent for each meal
time_spent_breakfast = sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * alpha_b[i] for i, recipe in enumerate(df_recipe_cost['Name']))
time_spent_lunch = sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * alpha_l[i] for i, recipe in enumerate(df_recipe_cost['Name']))
time_spent_dinner = sum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * alpha_d[i] for i, recipe in enumerate(df_recipe_cost['Name']))

# Print out time spent for each meal
print("Time spent for breakfast:", round(time_spent_breakfast, 2), "minutes")
print("Time spent for lunch:", round(time_spent_lunch, 2), "minutes")
print("Time spent for dinner:", round(time_spent_dinner, 2), "minutes")



Status: 1
Objective: 37.0
Time spent for breakfast: 22.0 minutes
Time spent for lunch: 60.0 minutes
Time spent for dinner: 119.0 minutes


In [5]:
print("breakfast")
print(output_b)
print("lunch")
print(output_l)
print("dinner")
print(output_d)

breakfast
{'Slow Cooker Chicken and Dumplings': 2.0}
lunch
{'Broccolini Quinoa Pilaf': 2.0}
dinner
{'Broccolini Quinoa Pilaf': 1.0, 'Slow Cooker Chicken and Dumplings': 4.0, 'Vanilla Cream Cakes, Easy and Fluffy Holiday Cakes': 1.0}


Experiments with the exponential decay. non-linear program

In [6]:
#define decay factor and decay factor multiplier
decay=0.4
decay_mul=1-decay
# Define decision variables
recipes_to_produce = LpVariable.dicts("Recipe", df_recipe_cost['Name'], lowBound=0, cat='Integer')
is_selected = LpVariable.dicts("IsSelected", df_recipe_cost['Name'], cat='Binary')

# Define the optimization problem
prob = LpProblem("Recipe_Production", LpMaximize)

# Objective function: maximize food preference produced with preference decay for duplicated meals
prob += lpSum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Preference'].values[0] * ((1-decay_mul**recipes_to_produce[recipe])/(1-decay_mul)) for recipe in df_recipe_cost['Name'])

# Constraint 1: time cost
prob += lpSum(df_recipe_cost.loc[df_recipe_cost['Name'] == recipe, 'Time'].values[0] * recipes_to_produce[recipe] for recipe in df_recipe_cost['Name']) <= time_available

# Constraint 2: nutritional requirements
for nutrient in nutrient_requirements:
    prob += lpSum(df_recipe_nutrition.loc[df_recipe_nutrition['Name'] == recipe, nutrient].values[0] * recipes_to_produce[recipe] for recipe in df_recipe_nutrition['Name']) >= nutrient_requirements[nutrient]

# Solve the optimization problem
prob.solve()

# Print the results
print("Status:", prob.status)
print("Objective:", round(prob.objective.value(), 2))

# Initialize an empty list to store the quantities
alpha = []

# Iterate through all recipes
for recipe in df_recipe_cost['Name']:
    # Retrieve the quantity from the optimization results
    quantity = recipes_to_produce[recipe].value()
    # Append the quantity to the alpha list
    alpha.append(round(quantity, 2))

# Print the whole array of quantities
#print("Quantity array (alpha):", alpha)

output = dict(zip(list(df_recipe_cost['Name']), alpha))
output


TypeError: unsupported operand type(s) for ** or pow(): 'float' and 'LpVariable'

cannot solve such a problem with PULP, which is a linear program solver